# Durable Functions Client Test

This notebook sends a POST request to the local Azure Functions endpoint `http://localhost:7071/api/client` using the Python `requests` library.

Steps:
1. Define the JSON payload with a `records` array.
2. Send POST request; receive status URLs from Durable Functions (includes `statusQueryGetUri`).
3. Poll the status URL until the orchestration finishes.
4. Display final output.

Run the cells in order. Ensure the function host is running locally (e.g. via `func start` or your `startLocal.sh`).

In [ ]:
# Define the payload matching sampleJSONrequest.json
payload = {
      "name": "role_library-3.pdf",
      "container": "bronze"
}
payload

In [ ]:
import requests, json, time

FUNCTION_ENDPOINT = "http://localhost:7071/api/client"
# FUNCTION_ENDPOINT = "https://func-processing-yt7giyre5immc.azurewebsites.net/api/client?code="
# Send POST request to start orchestration
response = requests.post(FUNCTION_ENDPOINT, json=payload)
print("Status Code:", response.status_code)

if response.status_code != 202:
    print("Unexpected response:", response.text)
else:
    # Durable Functions returns JSON with status URLs
    start_info = response.json()
    # Display keys of interest
    for k in ["id", "statusQueryGetUri", "sendEventPostUri", "terminatePostUri", "purgeHistoryDeleteUri"]:
        print(f"{k}: {start_info.get(k)}")

print(start_info)

In [ ]:
import requests, time, json

# Helper to poll the orchestration status until completion or timeout
TERMINAL_STATUSES = {"Completed", "Failed", "Terminated"}

def poll_status(status_url: str, interval_seconds: float = 2.0, timeout_seconds: float = 120.0):
    """Poll the Durable Functions statusQueryGetUri until a terminal status.
    Returns the final status document (dict) or raises TimeoutError.
    """
    start = time.time()
    attempt = 0
    while True:
        attempt += 1
        r = requests.get(status_url)
        if r.status_code != 200:
            print(f"Attempt {attempt}: Unexpected status code {r.status_code} -> {r.text[:200]}")
        else:
            status_doc = r.json()
            runtime_status = status_doc.get("runtimeStatus")
            print(f"Attempt {attempt}: runtimeStatus={runtime_status}")
            if runtime_status in TERMINAL_STATUSES:
                return status_doc
        if time.time() - start > timeout_seconds:
            raise TimeoutError(f"Polling exceeded {timeout_seconds} seconds without terminal status.")
        time.sleep(interval_seconds)

# Extract the status URL from the earlier cell output (start_info)
status_url = start_info.get("statusQueryGetUri")
if not status_url:
    raise ValueError("statusQueryGetUri not found in start_info. Run the previous cell first.")

final_status = poll_status(status_url)

print("\nFinal Status Document:\n")
print(json.dumps(final_status, indent=2))

# If Completed, the orchestrator's output is in 'output'
if final_status.get("runtimeStatus") == "Completed":
    print("\nOrchestrator Output:\n")
    print(json.dumps(final_status.get("output"), indent=2))
else:
    print("\nOrchestration did not complete successfully.")